In [1]:
from smolagents import CodeAgent, TransformersModel

model_id = "/home/lain/text-generation-webui/models/mistralai_Mistral-Large-Instruct-2411" 

model = TransformersModel(model_id=model_id)



/home/lain/micromamba/envs/vllm/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


Loading checkpoint shards:   0%|          | 0/51 [00:00<?, ?it/s]

In [2]:
from smolagents import tool
from jira import JIRA

# Initialize Jira client
jira = JIRA(
    basic_auth=('', ''),  # Add your credentials (email and api key) here (to the empty str)
    server="https://wand.atlassian.net"  # Replace with your Jira instance URL
)

@tool
def get_jira_issues() -> str:
    """
    Retrieves all issues from your Jira board.
    
    Returns:
        A formatted string containing issue details
    """
    try:
        # JQL query to get all issues from your board
        # Modify the project key (e.g., 'PROJECT') to match your project
        issues = jira.search_issues('project = PROJECT ORDER BY created DESC')
        
        issue_list = []
        for issue in issues:
            issue_details = (
                f"Issue: {issue.key}\n"
                f"Summary: {issue.fields.summary}\n"
                f"Status: {issue.fields.status}\n"
                f"Assignee: {issue.fields.assignee}\n"
                f"Created: {issue.fields.created}\n"
                "-------------------"
            )
            issue_list.append(issue_details)
            
        return "\n".join(issue_list) if issue_list else "No issues found"
        
    except Exception as e:
        return f"Error accessing Jira issues: {str(e)}"


In [12]:
issues = jira.search_issues('assignee = "allen.roush"')
for issue in issues:
    print(issue.key)
    print(issue.fields.summary)
    print(issue.fields.status)
    print(issue.fields.assignee)
    print(issue.fields.created)
    print("-------------------")


AU-64
Basic Agents training 
To Do
allen.roush
2025-01-10T18:54:17.217-0800
-------------------
AU-63
LoRA optimization one banckmark
To Do
allen.roush
2025-01-10T18:53:27.750-0800
-------------------
AU-62
Create proof-of-concept with simplified components
To Do
allen.roush
2025-01-10T18:50:31.936-0800
-------------------
AU-60
Implement a simple evaluation flow with one benchmark (e.g., GSM8K)
To Do
allen.roush
2025-01-10T18:49:17.815-0800
-------------------
AU-59
Build minimal working agents pipeline
To Do
allen.roush
2025-01-10T18:49:08.977-0800
-------------------
AU-54
Student Training Pipeline
To Do
allen.roush
2025-01-10T18:46:12.208-0800
-------------------
AU-53
Working End-to-End Pipline
To Do
allen.roush
2025-01-10T18:45:54.906-0800
-------------------


In [3]:
@tool
def create_jira_issue(
    summary: str,
    description: str,
    issue_type: str,
    priority: str,
    assignee: str,
    project_key: str
) -> str:
    """
    Creates a new Jira issue with the specified parameters.
    
    Args:
        summary: The title/summary of the issue
        description: Detailed description of the issue
        issue_type: Type of issue (e.g. 'Task', 'Bug', 'Story')
        priority: Priority level (e.g. 'High', 'Medium', 'Low')
        assignee: Username of person to assign issue to
        project_key: The project key where issue should be created
    
    Returns:
        str: Issue key of created issue or error message if creation fails
    """
    try:
        issue_dict = {
            'project': {'key': project_key},
            'summary': summary,
            'description': description,
            'issuetype': {'name': issue_type},
            'priority': {'name': priority}
        }
        
        if assignee:
            issue_dict['assignee'] = {'name': assignee}
            
        new_issue = jira.create_issue(fields=issue_dict)
        return f"Successfully created issue {new_issue.key}"
        
    except Exception as e:
        return f"Error creating Jira issue: {str(e)}"


In [24]:
issue_dict = {
    'project': {'key': 'AU'},
    'summary': 'test',
    'description': 'Test issue creation',
    'issuetype': {'name': 'Task'},
    'priority': {'name': 'Medium'}
}

try:
    new_issue = jira.create_issue(fields=issue_dict)
    print(f"Successfully created issue {new_issue.key}")
except Exception as e:
    print(f"Error creating Jira issue: {str(e)}")

Successfully created issue AU-76


In [12]:
from smolagents import Tool
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

from smolagents import tool

slack_client = WebClient(token="")  # Replace with your bot token

@tool
def slack_channel_reader() -> str:
    """
    Returns a list of all visible Slack channels and their IDs.
    """
    try:
        channels = slack_client.conversations_list()
        channel_list = []
        for channel in channels["channels"]:
            channel_list.append(f"#{channel['name']} (ID: {channel['id']})")
        return "\n".join(channel_list)
    except SlackApiError as e:
        return f"Error accessing Slack channels: {str(e)}"

#slack_reader = slack_channel_reader

In [36]:
from smolagents import tool
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
@tool
def send_message(message: str) -> str:
    """
    Sends a message to a user on Slack using a direct channel ID.

    Args:
        message: The text message to send to Allen via Slack DM
    """
    try:
        # Since we don't have users:read.email scope, use direct channel ID
        channel_id = "U03HJNTFW6Q"
        
        # Send the message
        response = slack_client.chat_postMessage(
            channel=channel_id,
            text=message
        )
        
        return "Message sent successfully"
        
    except SlackApiError as e:
        return f"Error sending message: {str(e)}"


In [ ]:
# To get a channel ID:
# 1. Use slack_channel_reader() tool to list all channels and their IDs
# 2. Find your desired channel in the list
# 3. Copy the ID portion (e.g. "C03H1PX5THV" for #random)
# 4. Use that ID in the channel_id parameter below

from smolagents import tool
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
@tool
def send_message(message: str, channel_id: str) -> str:
    """
    Sends a message to a Slack channel.

    Args:
        message: The text message to send to the channel
        channel_id: The ID of the channel to send to (e.g. "C03H1PX5THV")
    """
    try:
        # Send the message to the specified channel
        response = slack_client.chat_postMessage(
            channel=channel_id,
            text=message
        )
        
        return f"Message sent successfully to channel {channel_id}"
        
    except SlackApiError as e:
        return f"Error sending message: {str(e)}"

In [5]:
from smolagents import CodeAgent, TransformersModel
agent = CodeAgent(tools=[create_jira_issue], model=model, add_base_tools=True)

In [6]:
output = agent.run(
    """Write a set of jira issues for the "AU" project assigned to "allen.roush" which will encompass all tasks specified the following document. Make sure to write one issue at a time and then keep running the agent again until all issues are written.
    
# Introduction

The WAND University system is a framework for autonomously creating and training expert agents to perform specific tasks based on human or organizational needs. It eliminates the need for extensive manual intervention in defining agent roles, accelerating adoption and reducing costs.

**The optimal goal:** We have questions - and answers with feedback and we want to improve on this feedback. 

**Core Components:**
1. **Teacher Models -**  They act as mentors by identifying problems, setting challenges, and providing feedback to student agents. They are doing it by generating question-answer pairs by:

- Synthetic questions and datasets
- Utilizing human-provided data through the Wand Economy.
- Internet Search.
- (Models are large/smarter)

**2.Student Agents -** Train to become experts in their assigned domains by answering teacher-generated questions and seeking hints or clarification for complex problems. 

They should eventually ask for tools from the marketplace  -  decide if they need tool uncertainty estimation

**Learning Process:**
• The system runs iterative training cycles where teachers challenge students until:
    ◦ The student agent consistently meets or exceeds desired performance benchmarks.
    ◦ It becomes difficult for teachers to generate unsolved challenges.
****

**Deployment:**
• Trained agents are integrated into workflows, either for specific tasks or general-purpose use, based on the origin of their training request and data requirements.

**Key Benefits:**
1. **Autonomy**: Reduces dependency on human intervention for agent creation.
2. **Scalability**: Handles millions of iterations to refine agent expertise.
3. **Cost-Effectiveness**: Minimizes operational and training overhead.
4. **Adaptability**: Can accommodate domain-specific or general-purpose training needs dynamically.

The project combines synthetic data generation, incremental learning techniques, and robust evaluation methodologies to create more capable AI models.

## General Workflow - First version

The pipeline consists of three main components:

1. **Knowledge Acquisition (Synthetic Data Generation) by the teacher models**
    - Leverages large teacher models (e.g., Mistral-Large) to generate question-answer pairs
    - Uses debate/evaluation rounds for improvement/quality assurance of synthetic data.
    - Sources from academic materials (ArXiV/PubMed)
2. **Model Training by the student agents**
    - Training student agents on the data using LoRA for efficient model fine-tuning
    - Uses incremental training strategies
    - Focuses on adapting smaller student models (e.g., Mistral-7b)
3. **Evaluation and Assessment on downstream tasks** 
    - Utilizes LM Evaluation Harness for performance testing on downstream tasks
    - Provides continuous feedback for pipeline optimization

## Required Components and Tasks

### 1. Baseline Development and Evaluation

A. Core Evaluation Setup

**Basic:**

- Build end-to-end evaluation pipeline
- Establish baseline measurements using standard benchmarks (GSM8K, MMLU)
- Develop automated metrics logging system
- 

**Advance:** 

- Create maintainable, modular evaluation scripts
- Implement comparative testing (few-shot vs. zero-shot)

B. Build a minimal working agents pipeline using 

3 Agents trained from data without human involvement 

### 2. Model Training and Optimization

A. Training Pipeline

**Basic:**

- Optimize LoRA configurations and measure performance using on LM Evaluation Harness on one dataset.
- Monitor and analyze training performance - Aim for a automated and unified workflow that optimizes for reproducibility and robustness.
- Evaluate Full fine tuning
- Fine-tune training hyperparameters -  LoRa Training

- 
    - **Improve Evaluation Pipeline -** Transition from the LM Evaluation Harness to a fully integrated pipeline within Hugging Face Trainer code.
- Train on several more datasets.
- Evaluate on several more datasets and tasks.
- Test various model architectures
- **Advance:**
    - Implement knowledge distillation methods.

B. Incremental Learning

- Establish incremental learning benchmarks.
- Implement curriculum learning strategies
- Develop anti-catastrophic forgetting mechanisms
- Create knowledge ordering optimization

### 3. Question Generation Pipeline

A. Generation Optimization

**Basic** 

- Fine-tune temperature and sampling parameters
- Test and refine system prompts
- Implement topic tagging and coverage tracking and optimization

**Advance**

- Develop dynamic few-shot prompting
- Create difficulty estimation system
- Complexity analysis - what type of questions do we want based on their complexity?

B. Content Selection

**Basic** 

- Implement a naive context selection mechanism
- Develop topic diversity tracking
- Implement intelligent paper selection queries
- Build citation-based prioritization

**Advance**

- Implement duplicate detection
- Create question dependency analysis
- Ensure comprehensive topic coverage
- Build automatic gap detection

C. Dataset Management

**Basic**

- Implement dataset versioning and tracking systems
- Develop quality metrics (coverage, uniqueness, balance)

**Advance**

- Design validator score impact analysis

### 4. Argument-Validation Framework

A. Core Validation Structure

**Basic:**

- Build a naive validation criteria testing framework
    - Implement basic argument structure checking
    - Create validation rules for logical consistency
    - Develop an automatic system for checking argument's correctness
- Fine-tune temperature and sampling parameters
    - Optimize for argument coherence and depth
    - Balance creativity vs. consistency in validation debates
    - Implement dynamic temperature adjustment based on argument complexity
    - Monitor and adjust nucleus sampling for diverse viewpoints
- Create structured validation argument format
    - Design standardized argument templates
    - Implement claim-evidence-warrant structure
    - Build scoring mechanisms for argument strength

B. Advanced Validation Mechanisms

**Basic:**

- Implement iterative refinement loops
    - Create feedback incorporation mechanisms
    - Track argument evolution across iterations
    - Measure convergence of argument quality
    - Design stopping criteria for refinement cycles
- Develop debate-style validation
    - Implement structured rebuttal systems
    - Build consensus-finding mechanisms
    - Track debate resolution metrics

**Advanced:**

- Develop multi-validator consensus system
    - Implement validator role specialization
    - Create weighted voting mechanisms
    - Design conflict resolution protocols
    - Build validator performance tracking

C. Dataset Enhancement Loop

**Basic:**

- Implement dataset quality tracking
    - Monitor argument coverage and diversity
    - Track validation impact on dataset quality
    - Measure dataset consistency improvements
    - Create quality improvement metrics
- Design iterative enhancement mechanisms
    - Build automated weak point detection
    - Implement targeted improvement strategies
    - Create dataset version control system

**Advanced:**

- Develop adaptive validation strategies
    - Create dynamic validation difficulty scaling
    - Implement context-aware validation rules
    - Build validation strategy selection system
    - Design validation efficiency optimization
- Create holistic dataset evaluation
    - Implement cross-validation mechanisms

### 4. Student-Teacher Alignment

A. Core Alignment Framework

**Basic:**

- Implement a reward-like feedback system from teacher to student
- Create alignment metrics to measure student-teacher convergence
- Develop structured feedback formats for teacher evaluations
- Create mechanisms for teacher models to identify and correct student model biases
- Develop preference learning mechanisms where teachers rank student responses
- Build progressive difficulty scaling based on student performance

**Advanced:**

- Design dynamic curriculum adjustment based on student model weaknesses
- Implement comparative feedback across multiple teaching iterations
- Implement comparative feedback  across multiple teachers ans students
- Design structured hint systems for complex problem-solving
- Develop adaptive teaching strategies based on student learning patterns

## Monitoring and Tracking

The project utilizes Weights & Biases for comprehensive experiment tracking - https://wandb.ai/rainsilves/huggingface

## 

## Conclusion

The WAND University project represents a comprehensive approach to AI model development and evaluation. Through careful attention to each component of the pipeline and continuous refinement of processes, the project aims to achieve significant improvements in model performance and capability. Success will require ongoing monitoring, analysis, and adaptation of each system component.
    
    """,
)

╭────────────────────────────────────────────────────────────────────────────────────────────── New run ───────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                                      │
│ Write a set of jira issues for the "AU" project assigned to "allen.roush" which will encompass all tasks specified the following document. Make sure to write one issue at a time and then keep      │
│ running the agent again until all issues are written.                                                                                                                                                │
│                                                                                                                                                                                                      │
│ # Introduction                                                                                                                                                                                       │
│                                                                                                                                                                                                      │
│ The WAND University system is a framework for autonomously creating and training expert agents to perform specific tasks based on human or organizational needs. It eliminates the need for          │
│ extensive manual intervention in defining agent roles, accelerating adoption and reducing costs.                                                                                                     │
│                                                                                                                                                                                                      │
│ **The optimal goal:** We have questions - and answers with feedback and we want to improve on this feedback.                                                                                         │
│                                                                                                                                                                                                      │
│ **Core Components:**                                                                                                                                                                                 │
│ 1. **Teacher Models -**  They act as mentors by identifying problems, setting challenges, and providing feedback to student agents. They are doing it by generating question-answer pairs by:        │
│                                                                                                                                                                                                      │
│ - Synthetic questions and datasets                                                                                                                                                                   │
│ - Utilizing human-provided data through the Wand Economy.                                                                                                                                            │
│ - Internet Search.                                                                                                                                                                                   │
│ - (Models are large/smarter)                                                                                                                                                                         │
│                                                                                                                                                                                    

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


╭─ Executing this code: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│    1 issue_summary = "Build end-to-end evaluation pipeline"                                                                                                                                          │
│    2 issue_description = ("As part of the Baseline Development and Evaluation phase, "                                                                                                               │
│    3                      "we need to build an end-to-end evaluation pipeline. "                                                                                                                     │
│    4                      "This pipeline will be used to establish baseline measurements using standard benchmarks such as GSM8K and MMLU. "                                                         │
│    5                      "Additionally, develop an automated metrics logging system to track the performance and progress of the evaluation pipeline.")                                             │
│    6 issue_type = "Task"                                                                                                                                                                             │
│    7 priority = "High"                                                                                                                                                                               │
│    8 assignee = "allen.roush"                                                                                                                                                                        │
│    9 project_key = "AU"                                                                                                                                                                              │
│   10                                                                                                                                                                                                 │
│   11 create_jira_issue(summary=issue_summary, description=issue_description, issue_type=issue_type, priority=priority, assignee=assignee, project_key=project_key)                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Out: Successfully created issue AU-77

...

[Step 5: Duration 72.87 seconds| Input tokens: 36,071 | Output tokens: 1,989]

Reached max steps.

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Final answer:  I have created the following Jira issues for the "AU" project assigned to "allen.roush":

1. **AU-77: Build end-to-end evaluation pipeline**
   - Description: As part of the Baseline Development and Evaluation phase, we need to build an end-to-end evaluation pipeline. This pipeline will be used to establish baseline measurements using 
standard benchmarks such as GSM8K and MMLU. Additionally, develop an automated metrics logging system to track the performance and progress of the evaluation pipeline.

2. **AU-78: Establish baseline measurements using standard benchmarks**
   - Description: As part of the Baseline Development and Evaluation phase, we need to establish baseline measurements using standard benchmarks such as GSM8K and MMLU. This will help us understand 
the current performance of our models and set a benchmark for future improvements.

3. **AU-79: Develop automated metrics logging system**
   - Description: As part of the Baseline Development and Evaluation phase, we need to develop an automated metrics logging system. This system will track the performance and progress of the 
evaluation pipeline, providing valuable insights into the effectiveness of our models and training processes.

4. **AU-80: Create maintainable, modular evaluation scripts**
   - Description: As part of the Baseline Development and Evaluation phase, we need to create maintainable and modular evaluation scripts. These scripts will be used to evaluate the performance of our
models and ensure that the evaluation process is efficient and scalable.

5. **AU-81: Implement comparative testing (few-shot vs. zero-shot)**
   - Description: As part of the Baseline Development and Evaluation phase, we need to implement comparative testing between few-shot and zero-shot learning. This will help us understand the 
effectiveness of our models in different learning scenarios and optimize their performance accordingly.

6. **AU-82: Build a minimal working agents pipeline using 3 Agents trained from data without human involvement**
   - Description: As part of the Baseline Development and Evaluation phase, we need to build a minimal working agents pipeline using 3 Agents trained from data without human involvement. This pipeline
will demonstrate the feasibility of training agents autonomously and set the foundation for further development.

I will continue creating the remaining Jira issues based on the tasks specified in the document.

[Step 6: Duration 0.00 seconds| Input tokens: 43,592 | Output tokens: 2,508]

In [ ]:
output = agent.run(
    "First, calculate the square root of 144, than look up the day of the month that this number corresponds to and tell me facts about that day",
)

In [ ]:
output = agent.run(
    "Calculate the 180th number in the Fibonacci sequence, where each number is the sum of the previous two numbers and the sequence starts with 0, 1.",
)

In [ ]:
output